In [1]:
import pandas as pd
import geopandas as gpd
import censusdata
import numpy as np

In [ ]:
# Downloading Census block boundaries for Chicago
census_gdf = gpd.read_file("https://data.cityofchicago.org/resource/bt9m-d2mf.geojson?$limit=9999999")
census_gdf["GEOID"] = census_gdf["geoid10"].map(lambda x: str(x)[:12])
census_group = census_gdf.dissolve(by='GEOID').reset_index()
census_group.crs = "EPSG:4326"
census_group['GEOID'] = census_group['GEOID'].astype(int)

In [ ]:
# Housing Assessment Data by Census Block Group
# This dataset was previously merged with block group-level ACS data.
housing = pd.read_csv('data/final_assessments_block_group.csv', sep='\t', index_col=0)
merged = census_group.merge(housing, on ="GEOID", how="inner")
merged.head(2)

In [ ]:
# Water Quality Data
wdf_g = gpd.read_file('data/water_quality_clean.shp')
wdf_g.crs = "EPSG:4326"
wdf_g.head(2)
wdf_g.columns = ['date', 'address', '1min_draw', '3min_draw', '5min_draw', 
                 'location', 'lat', 'long', 
                 'avg_reading', 'max_reading', 't_high', 't_med', 'geometry']

# Merging datasets
water_acs = gpd.sjoin(wdf_g, merged, how='inner', op='within')

In [ ]:
# Creating aggregate dataset
avg_reading = water_acs.groupby('GEOID')['avg_reading'].mean().to_frame('avg_reading_all').reset_index()
max_reading = water_acs.groupby('GEOID')['max_reading'].mean().to_frame('max_reading_all').reset_index()
sample_cnt = water_acs.groupby('GEOID').size().to_frame('sample_cnt').reset_index()

for_plot = census_group.merge(avg_reading, on="GEOID", how="outer")
for_plot = for_plot.merge(max_reading, on="GEOID", how="outer")
for_plot = for_plot.merge(sample_cnt, on="GEOID", how="outer")
for_plot = for_plot.merge(housing, on="GEOID", how="outer")

for_plot['sample_cnt'] = for_plot['sample_cnt'].fillna(0)
for_plot['samp_density'] = for_plot['sample_cnt']/for_plot['tot_pop']
for_plot['samp_density'] = for_plot['samp_density'].replace(np.infty, 0)
for_plot['samp_density'] = for_plot['samp_density'].fillna(0)
for_plot['threshold_high'] = np.where(for_plot['max_reading_all'] >= 15, 1, 0)
for_plot['threshold_medium'] = np.where(for_plot['max_reading_all'] >= 5, 1, 0)

In [ ]:
fig = for_plot.plot(column='sample_cnt',
                        cmap = 'Greys',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig 1. Sampling Density by Block Group', 
              fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
fig = for_plot.plot(column='samp_density',
                        cmap = 'Greys',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig 1. Sampling Density by Block Group', 
              fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
fig = for_plot.plot(column='max_reading_all',
                        cmap = 'BuPu',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig 2. Mean Lead Sample Reading by \n Census Block Group (ppb)', 
              fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
test = water_acs.groupby('GEOID')['max_reading'].apply(lambda x: (x>=15).sum()).to_frame('15cnt').reset_index()
test1 = water_acs.groupby('GEOID')['max_reading'].apply(lambda x: (x>=5).sum()).to_frame('5cnt').reset_index()
test2 = for_plot.merge(test, on="GEOID", how="outer")
test2 = test2.merge(test1, on="GEOID", how="outer")

fig = test2.plot(column='15cnt',
                        cmap = 'BuPu',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig X. Number of Houses Exceeding 15 ppb by \n Census Block Group', 
              fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
fig = test2.plot(column='5cnt',
                        cmap = 'BuPu',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig X. Number of Houses Exceeding 5 ppb by \n Census Block Group', 
              fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
fig = for_plot.plot(column='perc_black',
                        cmap = 'Blues',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig 3. Total Population (%Black/African American) \n by Census Block Group', 
              fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
fig = for_plot.plot(column='med_income',
                        cmap = 'Greens',
                        figsize=(11,7),   
                        scheme='natural_breaks', k=4, 
                        legend = True,
                        edgecolor='darkgrey',
                        alpha=0.8)
    
fig.set_title('Fig 5. Median Income by Census Block Group', fontdict= {'fontsize':12})
fig.set_axis_off()
fig.get_legend().set_bbox_to_anchor((0., 0., 0.2, 0.2))

for legend_handle in fig.get_legend().legendHandles:  
    legend_handle._legmarker.set_markeredgewidth(1)
    legend_handle._legmarker.set_markeredgecolor('k')

In [ ]:
# Descriptive Statistics
import seaborn as sns
import matplotlib.pyplot as plt

print("The range for the water quality dataset is:")
print("Max value: {}\nMin value: {}".format(
    max(wdf_g['max_reading']), min(wdf_g['max_reading'])))

# Data is right-skewed
sampl_distrib = sns.boxplot(x="max_reading_all", data=for_plot)
plt.title('Fig X. Lead Levels (ppb)', fontsize=13) 
plt.xlabel('Lead Level in Sample (ppb)', fontsize=11)
plt.tight_layout()

In [ ]:
 # Threshold Categorical Variable

def categories(x): 
    if 0 < x < 5:
        return 'low'
    if 5 <= x < 15: 
        return 'medium'
    elif x >= 15:
        return 'high'
    
water_acs['lead_cat'] = water_acs['max_reading'].apply(categories)

med_income = sns.catplot(x="lead_cat", y="med_income", kind='box', data=water_acs, 
                         order=["low", "medium", "high"]) 
labels = med_income.set(xlabel='Lead Reaing Category', ylabel='Median Income',
title='Median Income Distribution by Lead Reading Category')

In [ ]:
perc_black = sns.catplot(x="lead_cat", y="perc_black", kind='box', data=water_acs, 
                         order=["low", "medium", "high"]) 
labels = perc_black.set(xlabel='Lead Reading Category', 
                        ylabel='Population (% Black/African American)',
                        title='Population (% Black) by Lead Reading Category')

In [ ]:
perc_nwhite = sns.catplot(x="lead_cat", y="perc_non_w", kind='box', data=water_acs, 
                         order=["low", "medium", "high"]) 
labels = perc_nwhite.set(xlabel='Lead Reading Category', 
                        ylabel='Population (% Non-White)',
                        title='Population (% Non-White) by Lead Reading Category')

In [ ]:
# sns.scatterplot(data=for_plot, x="med_income", y="max_reading_all", 
#                hue="threshold_high", alpha=0.6)

ax = sns.scatterplot(x="med_income", y="max_reading_all",
                     data=for_plot[for_plot.threshold_high == 0], alpha=0.5)
sns.scatterplot(x="med_income", y="max_reading_all", for_plot=for_plot[for_plot.threshold_high == 1.0], 
                alpha=1.0, ax=ax)

In [ ]:
final_df = for_plot.drop(['geometry', 'statefp10', 'name10', 'blockce10', 'tract_bloc',
       'geoid10', 'tractce10', 'countyfp10', 'samp_density'], axis=1)
final_df.to_csv('data/final_df.csv')

In [ ]:
################################
####### ARCHIVE -- TO DELETE ###

In [ ]:
# Downloading Census block boundaries for Chicago
census_gdf = gpd.read_file("https://data.cityofchicago.org/resource/bt9m-d2mf.geojson?$limit=9999999")
census_gdf["geo_12"] = census_gdf["geoid10"].map(lambda x: str(x)[:12])
census_group = census_gdf.dissolve(by='geo_12').reset_index()
census_group.crs = "EPSG:4326"

# Pulling ACS data 
census_tables = {
    'GEO_ID': 'GEO_ID', 
    'B02001_001E': 'Race Total', 
    'B02001_002E': 'White', 
    'B02001_003E': 'Black', 
    'B19013_001E': 'Median Income',
    'B03002_002E': 'Not Hispanic'}

acs_df = censusdata.download("acs5", 
                              2018, 
                              censusdata.censusgeo([("state", "17"), 
                                                    ("county", "031"),
                                                    ("tract", "*"),
                                                    ("block group", "*")]), 
                              list(census_tables.keys()))

# Rename columns 
acs_df.rename(columns=census_tables, inplace=True)
acs_df["geo_12"] = acs_df["GEO_ID"].map(lambda x: str(x)[-12:])
merged = census_group.merge(acs_df, on ="geo_12", how="inner")

# Creating new attributes
merged["p_white"] = (merged["White"]/merged["Race Total"])*100
merged["p_black"] = (merged["Black"]/merged["Race Total"])*100
merged["p_hispanic"] = ((merged['Race Total']-merged['Not Hispanic'])/merged["Race Total"])*100

merged.rename(mapper={'Median Income':'med_income'}, axis='columns', inplace=True)
merged["med_income"].replace({-666666666.0: merged["med_income"].median()}, inplace=True)